In [22]:
import tensorflow as tf
from tfx import v1 as tfx

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

% load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

print('TFX version: {}'.format(tfx.__version__))
print('TensorFlow version: {}'.format(tf.__version__))

The tfx.orchestration.experimental.interactive.notebook_extensions.skip extension is already loaded. To reload it, use:
  %reload_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip
TFX version: 1.3.3
TensorFlow version: 2.6.2


In [23]:
import os
from absl import logging

PIPELINE_NAME = "iris"
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

DATA_ROOT = os.path.join("data")

logging.set_verbosity(logging.INFO)

# Retrieve data

In [24]:
import urllib.request

_data_url = 'https://datahub.io/machine-learning/iris/r/iris.csv'
_data_filepath = os.path.join(DATA_ROOT, "data.csv")

urllib.request.urlretrieve(_data_url, _data_filepath)

!head {_data_filepath}

sepallength,sepalwidth,petallength,petalwidth,class
5.1,3.5,1.4,0.2,Iris-setosa
4.9,3.0,1.4,0.2,Iris-setosa
4.7,3.2,1.3,0.2,Iris-setosa
4.6,3.1,1.5,0.2,Iris-setosa
5.0,3.6,1.4,0.2,Iris-setosa
5.4,3.9,1.7,0.4,Iris-setosa
4.6,3.4,1.4,0.3,Iris-setosa
5.0,3.4,1.5,0.2,Iris-setosa
4.4,2.9,1.4,0.2,Iris-setosa


# Create TFX pipeline and run it

In [19]:
_module_file = 'iris_utils.py'

In [20]:
def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     module_file: str, serving_model_dir: str,
                     metadata_path: str) -> tfx.dsl.Pipeline:
    """Implements the penguin pipeline with TFX."""
    # Brings data into the pipeline or otherwise joins/converts training data.
    example_gen = tfx.components.CsvExampleGen(
        input_base=data_root)

    # Computes statistics over data for visualization and example validation.
    statistics_gen = tfx.components.StatisticsGen(
        examples=example_gen.outputs['examples'])

    # Generate a schema based on your data statistics.
    # A schema defines the expected bounds, types, and properties of the features in your dataset.
    schema_gen = tfx.components.SchemaGen(
        statistics=statistics_gen.outputs['statistics'], infer_feature_shape=True)

    # Performs anomaly detection based on statistics and data schema.
    example_validator = tfx.components.ExampleValidator(
        statistics=statistics_gen.outputs['statistics'],
        schema=schema_gen.outputs['schema'])

    # Transforms input data using preprocessing_fn in the 'module_file'.
    transform = tfx.components.Transform(
        examples=example_gen.outputs['examples'],
        schema=schema_gen.outputs['schema'],
        materialize=False,
        module_file=module_file)

    # Uses user-provided Python function that trains a model.
    trainer = tfx.components.Trainer(
        module_file=module_file,
        examples=example_gen.outputs['examples'],
        schema=schema_gen.outputs['schema'],
        transform_graph=transform.outputs['transform_graph'],
        train_args=tfx.proto.TrainArgs(num_steps=2000),
        eval_args=tfx.proto.EvalArgs(num_steps=5))

    # Pushes the model to a filesystem destination.
    pusher = tfx.components.Pusher(
        model=trainer.outputs['model'],
        push_destination=tfx.proto.PushDestination(filesystem=tfx.proto.PushDestination
                                                   .Filesystem(base_directory=serving_model_dir))
    )

    components = [
        example_gen,
        statistics_gen,
        schema_gen,
        example_validator,
        transform,
        trainer,
        pusher,
    ]

    return tfx.dsl.Pipeline(
        pipeline_name=pipeline_name,
        pipeline_root=pipeline_root,
        metadata_connection_config=tfx.orchestration.metadata.sqlite_metadata_connection_config(metadata_path),
        components=components)

In [21]:
pipeline = _create_pipeline(
    pipeline_name=PIPELINE_NAME,
    pipeline_root=PIPELINE_ROOT,
    data_root=DATA_ROOT,
    module_file=_module_file,
    serving_model_dir=SERVING_MODEL_DIR,
    metadata_path=METADATA_PATH
)

tfx.orchestration.LocalDagRunner().run(pipeline)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Generating ephemeral wheel package for '/home/ekrem/Workplace/Python/tfx_seminar/iris_utils.py' (including modules: ['tfx_utils', 'iris_utils']).
INFO:absl:User module package has hash fingerprint version 6141aeec78e743e9b4d4c7b8c1a5f2ed2dbc4e6c5e44c7f22a6ab283e1dfa661.
INFO:absl:Executing: ['/home/ekrem/.pyenv/versions/3.8.7/envs/tfx/bin/python3.8', '/tmp/tmphkt0ct32/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmp_0t7rlxa', '--dist-dir', '/tmp/tmpb0uk_bbf']
/home/ekrem/.pyenv/versions/3.8.7/envs/tfx/lib/python3.8/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
INFO:absl:Successfully built user code wheel distribution at 'pipelines/iris/_w

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying tfx_utils.py -> build/lib
copying iris_utils.py -> build/lib
installing to /tmp/tmp_0t7rlxa
running install
running install_lib
copying build/lib/tfx_utils.py -> /tmp/tmp_0t7rlxa
copying build/lib/iris_utils.py -> /tmp/tmp_0t7rlxa
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmp_0t7rlxa/tfx_user_code_Transform-0.0+6141aeec78e743e9b4d4c7b8c1a5f2ed2dbc4e6c5e44c7f22a6ab283e1dfa661-py3.8.egg-info
running in

/home/ekrem/.pyenv/versions/3.8.7/envs/tfx/lib/python3.8/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
INFO:absl:Successfully built user code wheel distribution at 'pipelines/iris/_wheels/tfx_user_code_Trainer-0.0+6141aeec78e743e9b4d4c7b8c1a5f2ed2dbc4e6c5e44c7f22a6ab283e1dfa661-py3-none-any.whl'; target user module is 'iris_utils'.
INFO:absl:Full user module path is 'iris_utils@pipelines/iris/_wheels/tfx_user_code_Trainer-0.0+6141aeec78e743e9b4d4c7b8c1a5f2ed2dbc4e6c5e44c7f22a6ab283e1dfa661-py3-none-any.whl'
INFO:absl:Using deployment config:
 executor_specs {
  key: "CsvExampleGen"
  value {
    beam_executable_spec {
      python_executor_spec {
        class_path: "tfx.components.example_gen.csv_example_gen.executor.Executor"
      }
    }
  }
}
executor_specs {
  key: "ExampleValidator"
  value {
    python_class_executable_spec {
      class_path: "tfx

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying tfx_utils.py -> build/lib
copying iris_utils.py -> build/lib
installing to /tmp/tmpuxdpzg35
running install
running install_lib
copying build/lib/tfx_utils.py -> /tmp/tmpuxdpzg35
copying build/lib/iris_utils.py -> /tmp/tmpuxdpzg35
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /tmp/tmpuxdpzg35/tfx_user_code_Trainer-0.0+6141aeec78e743e9b4d4c7b8c1a5f2ed2dbc4e6c5e44c7f22a6ab283e1dfa661-py3.8.egg-info
running install_scripts
crea

INFO:absl:Examples generated.
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 32 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'examples': [Artifact(artifact: uri: "pipelines/iris/CsvExampleGen/examples/32"
custom_properties {
  key: "input_fingerprint"
  value {
    string_value: "split:single_split,num_files:1,total_bytes:4753,xor_checksum:1637142150,sum_checksum:1637142150"
  }
}
custom_properties {
  key: "name"
  value {
    string_value: "iris:2021-11-17T10:42:32.692709:CsvExampleGen:examples:0"
  }
}
custom_properties {
  key: "span"
  value {
    int_value: 0
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "1.3.3"
  }
}
, artifact_type: name: "Examples"
properties {
  key: "span"
  value: INT
}
properties {
  key: "split_names"
  value: STRING
}
properties {
  key: "version"
  value: INT
}
)]}) for execution 32
INFO:absl:MetadataStore with DB connection 

...

INFO:tensorflow:struct2tensor is not available.
INFO:absl:Feature class has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature petallength has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature petalwidth has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature sepallength has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature sepalwidth has a shape dim {
  size: 1
}
. Setting to DenseTensor.
2021-11-17 10:42:48.465506: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-11-17 10:42:48.465563: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-11-17 10:42:48.466919: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1614] Profiler found 1 GPUs
2021-11-17 10:42:48.590489: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-11-17 10:42:48.592600: I tensorflow/core/profiler/internal/gpu/cupti_tracer.

   4/2000 [..............................] - ETA: 2:19 - loss: 1.4543 - sparse_categorical_accuracy: 0.2000 

2021-11-17 10:42:49.703829: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-11-17 10:42:49.703875: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-11-17 10:42:49.840413: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-11-17 10:42:49.841452: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1748] CUPTI activity buffer flushed
2021-11-17 10:42:49.861855: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:673]  GpuTracer has collected 86 callback api events and 83 activity events. 
2021-11-17 10:42:49.864763: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-11-17 10:42:49.870983: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: pipelines/iris/Trainer/model_run/37/train/plugins/profile/2021_11_17_10_42_49

2021-11-17 10:42:49.872992: I tensorflow/core/profiler/rpc/client/save_prof

2000/2000 [==============================] - 7s 3ms/step - loss: 0.0240 - sparse_categorical_accuracy: 0.9904 - val_loss: 2.3640 - val_sparse_categorical_accuracy: 0.9000

INFO:tensorflow:Assets written to: pipelines/iris/Trainer/model/37/Format-Serving/assets
INFO:absl:Training complete. Model written to pipelines/iris/Trainer/model/37/Format-Serving. ModelRun written to pipelines/iris/Trainer/model_run/37
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 37 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'model_run': [Artifact(artifact: uri: "pipelines/iris/Trainer/model_run/37"
custom_properties {
  key: "name"
  value {
    string_value: "iris:2021-11-17T10:42:32.692709:Trainer:model_run:0"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "1.3.3"
  }
}
, artifact_type: name: "ModelRun"
)], 'model': [Artifact(artifact: uri: "pipelines/iris/Trainer/model/37"
custom_properties {
  key: "name"
  value {
    string_value: "iris:2021-11-17T10:42:32.692709:Trainer:model:0"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    strin

# Visualize pipeline

In [21]:
from tfx.orchestration.portable.mlmd import execution_lib
from tfx.orchestration.metadata import Metadata
from tfx.orchestration.experimental.interactive import visualizations
from ml_metadata.proto import metadata_store_pb2


def get_latest_artifact(component_id):
    metadata_connection_config = tfx.orchestration.metadata.sqlite_metadata_connection_config(METADATA_PATH)

    with Metadata(metadata_connection_config) as metadata_handler:
        context = metadata_handler.store.get_context_by_type_and_name('node', f'{PIPELINE_NAME}.{component_id}')
        executions = metadata_handler.store.get_executions_by_context(context.id)
        latest_execution = max(executions, key=lambda e: e.last_update_time_since_epoch)

        return execution_lib.get_artifacts_dict(metadata_handler, latest_execution.id, metadata_store_pb2.Event.OUTPUT)


def visualize(artifact):
    visualization = visualizations.get_registry().get_visualization(artifact.type_name)
    visualization.display(artifact)


from tfx.orchestration.experimental.interactive import standard_visualizations

standard_visualizations.register_standard_visualizations()

## StatisticsGen

In [22]:
statistic_artifact = get_latest_artifact("StatisticsGen")['statistics'][0]

visualize(statistic_artifact)

INFO:absl:MetadataStore with DB connection initialized


## SchemaGen

In [23]:
schema_artifact = get_latest_artifact("SchemaGen")['schema'][0]

visualize(schema_artifact)

INFO:absl:MetadataStore with DB connection initialized


,Type,Presence,Valency,Domain
Feature name,,,,
'Petal.Length',FLOAT,required,,-
'Petal.Width',FLOAT,required,,-
'Sepal.Length',FLOAT,required,,-
'Sepal.Width',FLOAT,required,,-
'Species',STRING,required,,'Species'
'id',INT,required,,-


/home/ekrem/.pyenv/versions/3.8.7/envs/tfx/lib/python3.8/site-packages/tensorflow_data_validation/utils/display_util.py:180: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Values
Domain,
'Species',"'setosa', 'versicolor', 'virginica'"


## ExampleValidator

In [24]:
example_validator_articaft = get_latest_artifact("ExampleValidator")['anomalies'][0]

visualize(example_validator_articaft)

INFO:absl:MetadataStore with DB connection initialized


/home/ekrem/.pyenv/versions/3.8.7/envs/tfx/lib/python3.8/site-packages/tensorflow_data_validation/utils/display_util.py:217: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


## Transformer

In [25]:
# TODO

## Trainer

In [26]:
model_run_artifact_dir = get_latest_artifact("Trainer")['model_run'][0].uri

% load_ext tensorboard
% tensorboard --logdir {model_run_artifact_dir}

INFO:absl:MetadataStore with DB connection initialized


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Launching TensorBoard...